# SiRF Scanner

## Written by Michael George

Iterate through folders, looking for file reader issues caused by wrong extension, missing headers, bad checksums, etc.

In [1]:
import os
import sys

import pprint
import traceback

import numpy as np

corePath = os.path.join('..', 'core')
if corePath not in sys.path:
    sys.path.extend([corePath])

from file_reader import getFileReader

## Main Function

In [2]:
def scanFiles():
    '''Iterate through session archive testing each GPS file'''

    rootDir = os.path.join(projdir, 'sessions')

    exceptions = {}
    sats = {}
    satMeans = {}
    rawSummaries = {}
    
    fileCount = 0
    
    for root, subDirs, files in os.walk(rootDir):
        for file in files:
            ext = os.path.splitext(file)[1].lower()
            
            if ext and ext in ('.sbn', '.sbp'):
                filePath = os.path.join(root, file)
                reader = getFileReader(filePath)
                try:
                    reader.load()
                    
                    # Record field summary
                    for field in reader.rawData.dtype.names:
                        pass
                        rawMin = reader.rawData[field].min()
                        rawMax = reader.rawData[field].max()
                        
                        if reader.device in rawSummaries:
                            if field in rawSummaries[reader.device]:
                                if rawMin < rawSummaries[reader.device][field]['min']:
                                    rawSummaries[reader.device][field]['min'] = rawMin
                                if rawMax > rawSummaries[reader.device][field]['max']:
                                    rawSummaries[reader.device][field]['max'] = rawMax
                            else:
                                rawSummaries[reader.device][field] = {}
                                rawSummaries[reader.device][field]['min'] = rawMin
                                rawSummaries[reader.device][field]['max'] = rawMax
                        else:
                            rawSummaries[reader.device] = {}
                            rawSummaries[reader.device][field] = {}
                            rawSummaries[reader.device][field]['min'] = rawMin
                            rawSummaries[reader.device][field]['max'] = rawMax

                    # Record satellites
                    if reader.device in sats:
                        sats[reader.device].append(reader.data['sat'])
                    else:
                        sats[reader.device] = [reader.data['sat']]

                    # Record mean
                    satMean = reader.data['sat'].mean()
                    if reader.device in satMeans:
                        satMeans[reader.device].append(satMean)
                    else:
                        satMeans[reader.device] = [satMean]

                    print('.', end='')

                except Exception:
                    exceptions[file] = traceback.format_exc()

                    print('E', end='')

                fileCount += 1

        if fileCount < 0:
            break
                
    # Show raw summaries
    print(os.linesep * 2 + 'Summaries:')
    pp = pprint.PrettyPrinter(indent=4)
    for device in rawSummaries:
        print(device)
        pp.pprint(rawSummaries[device])

    # Show stats
    print(os.linesep * 2 + 'Satellite Means:')
    pp = pprint.PrettyPrinter(indent=4)
    for device in sats:
        print(device)
        tmp = np.concatenate(sats[device], axis=0)
        unique, counts = np.unique(tmp, return_counts=True)
        print(np.asarray((unique, counts)).T)

    # Report errors
    print(os.linesep * 2 + 'Errors:')
    if len(exceptions) > 0:
        for file in exceptions:
            print(os.linesep + file)
            print(exceptions[file].split(os.linesep)[-2:-1])
    else:
        print(os.linesep + 'None!')

In [3]:
if __name__ == '__main__':
    projdir = os.path.realpath(os.path.join(sys.path[0], "..", ".."))

    scanFiles()    

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

[[      3     880]
 [      4    6965]
 [      5   12846]
 [      6  189078]
 [      7 1145378]
 [      8 2299911]
 [      9 3001855]
 [     10 2059938]
 [     11  938202]
 [     12  290394]]
GW-52
[[    3   107]
 [    4    90]
 [    5   146]
 [    6 10605]
 [    7 14760]
 [    8 41365]
 [    9 41028]
 [   10 28819]
 [   11   879]]
None
[[     3     21]
 [     4     84]
 [     5    314]
 [     6  12172]
 [     7  18501]
 [     8  37565]
 [     9  61481]
 [    10 109584]
 [    11  31928]
 [    12   1109]]
Unknown
[[  0   2]
 [  4   7]
 [  5   2]
 [  6  40]
 [  7 256]
 [  8  64]
 [  9 412]]


Errors:

None!
